In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

import sklearn
import scipy.sparse

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import seaborn as sns
from tqdm import tqdm_notebook

In [12]:
DATA_FOLDER = './readonly/'
RESULTS_FOLDER = './results/'

data = pd.read_excel(os.path.join(DATA_FOLDER, 'MFTRAVEL_HACKATHON.xlsx'), sheet_name=None)

In [13]:
countries = data['CountryDict']['CNTRY_CODE'].values.astype('int32')
data.keys()

odict_keys(['Data', 'CountryDict', 'FieldsDiscr'])

## ENV EDA

In [156]:
MaxFlightPrice = 100000
MaxHotelPrice = 100000
MaxHotelPrice = 100000
MaxRandDays = 100
MaxHotels = 500
MaxFlights = 100000

ZeroDate = pd.to_datetime('8/2/2020')

class Environment:
    def gen_dataset(self):
        k = dict()
        hotels = ['Baba', 'Bubu', 'Kata']
        hotel_ind = 0
        start_date = ZeroDate

        print('Generating country_to_hotels dataset...')
        
        for i in tqdm_notebook(countries):
            country_to_hotels = {'name' : [], 'day_price': [], 'available_from' : [], 'available_to' : []}

            for _ in range(MaxHotels):
                country_to_hotels['name'].append('{}_{}'.format(hotels[hotel_ind % 3], hotel_ind))
                country_to_hotels['day_price'].append(np.random.randint(MaxHotelPrice))

                delta1 = np.random.randint(MaxRandDays)
                delta2 = np.random.randint(MaxRandDays//2)
                country_to_hotels['available_from'].append(start_date + pd.DateOffset(days = delta1))
                country_to_hotels['available_to'].append(start_date + pd.DateOffset(days = delta1 + delta2))

                hotel_ind+=1

            k[i] = pd.DataFrame(data=country_to_hotels)
            
        self.country_to_hotels_ = k
        
    def __init__(self, country_to_hotels = None):
        print('Generating time_to_planes dataset...')
        
        d = {'time' : [], 'price': [], 'from': [], 'to' : []}
        start_date = ZeroDate
        
        for i in range(MaxFlights):
            if i % (MaxFlights // (MaxRandDays * 2)) == 0: 
                start_date += pd.DateOffset(days = 1)
                
            d['time'].append(start_date)
            d['price'].append(np.random.randint(MaxFlightPrice))
            d['from'].append(countries[np.random.randint(countries.size)])
            d['to'].append(countries[np.random.randint(countries.size)])

        self.planes_timetable_ = pd.DataFrame(data=d).set_index('time')
    
        if (country_to_hotels == None):
            self.gen_dataset()
        else:
            self.country_to_hotels_ = country_to_hotels
        
        
        print('Done')
        
    def get_tickets(self, time1, time2):
        return self.planes_timetable_[time1 : time2]
    
    def get_hotels(self, country_id, time1, time2): #datetime!!!
        return self.country_to_hotels_[country_id][
            (time1 > self.country_to_hotels_[country_id]['available_from']) &
            (time2 < self.country_to_hotels_[country_id]['available_to'])] 

In [149]:
save = env.country_to_hotels_

In [157]:
env = Environment(save)

Generating time_to_planes dataset...
Done


In [161]:
time1 = pd.to_datetime('28/8/2020')
time2 = pd.to_datetime('29/8/2020')

env.get_hotels(4, time1, time2)

,name,day_price,available_from,available_to
3,Kata_503,57931,2020-08-08,2020-09-26
6,Kata_506,84984,2020-08-06,2020-08-31
10,Baba_510,88833,2020-08-27,2020-09-19
23,Bubu_523,76752,2020-08-25,2020-09-06
43,Baba_543,27161,2020-08-25,2020-09-12
...,...,...,...,...
474,Kata_974,26532,2020-08-05,2020-09-14
475,Baba_975,41564,2020-08-24,2020-09-09
478,Baba_978,9088,2020-08-15,2020-09-07
486,Kata_986,13208,2020-08-15,2020-09-27


In [162]:
env.get_tickets(time1, time2)

,price,from,to
time,,,
2020-08-28,1519,554,208
2020-08-28,47774,104,104
2020-08-28,18892,520,404
2020-08-28,77592,232,64
2020-08-28,56376,344,-2147483648
...,...,...,...
2020-08-29,24364,-2147483648,876
2020-08-29,7228,694,860
2020-08-29,73598,4,694


## NN EDA

In [4]:
import tensorflow as tf
from sklearn import preprocessing

In [23]:
exp = data['Data'].copy()

def foo(x):
    if type(x) == int:
        return [x]
    return list(map(int, x.split('; ')))

exp['COUNTRIES_IN_TRIP'] = exp['COUNTRIES_IN_TRIP'].apply(foo)
# del data

In [24]:
exp['USING_INTERNET'] = exp['USING_INTERNET'].fillna(0).astype('bool')
le = preprocessing.LabelEncoder()

for column in ['GENDER', 'REGION', 'DEVICE_TYPE',
               'OS', 'SUBSAGE_MF_SEGMENT']:
    le.fit(list(exp[column].values))
    exp[column] = le.transform(list(exp[column].values))
    print('OKAY', column)

OKAY GENDER
OKAY REGION
OKAY DEVICE_TYPE
OKAY OS
OKAY SUBSAGE_MF_SEGMENT


In [26]:
# IT"S HEAVY
visited_country = dict()
for i in countries:
    list_t = []
    for j in exp['COUNTRIES_IN_TRIP']:
        list_t.append(i in j)
    
    visited_country[i] = list_t
    
exp = exp.join(pd.DataFrame(visited_country))
exp.drop(['COUNTRIES_IN_TRIP'], inplace = True, axis=1)

In [67]:
exp.columns[:19]

Index([           'User_ID',             'GENDER',                'AGE',
                   'REGION',        'DEVICE_TYPE',                 'OS',
       'SUBSAGE_MF_SEGMENT',     'USING_INTERNET',         'START_TRIP',
                 'END_TRIP',      'TRIP_DURATION',  'TRIP_MAIN_COUNTRY',
                  'ARPU_M3',      'SMS_IN_CNT_M3',     'SMS_OUT_CNT_M3',
        'MOU_IN_REVENUE_M3', 'MOU_OUT_REVENUE_M3',    'DOU_DURATION_M3',
                        895],
      dtype='object')

In [63]:
exp['User_ID'] = exp['User_ID'].astype('int32')
exp['AGE'] = exp['AGE'].fillna(-1).astype('int8')
exp['GENDER'] = exp['GENDER'].astype('int8')
exp['REGION'] = exp['REGION'].astype('int8')

In [71]:
exp[exp.columns[:19]]

,User_ID,GENDER,AGE,REGION,DEVICE_TYPE,OS,SUBSAGE_MF_SEGMENT,USING_INTERNET,START_TRIP,END_TRIP,TRIP_DURATION,TRIP_MAIN_COUNTRY,ARPU_M3,SMS_IN_CNT_M3,SMS_OUT_CNT_M3,MOU_IN_REVENUE_M3,MOU_OUT_REVENUE_M3,DOU_DURATION_M3,895
0,80000,2,48,55,5,0,1,True,2019-10-03,2019-10-07,5.0,222,3264.41,300,23,463.933332,408.483325,4263.106471,False
1,80000,2,48,55,5,0,1,True,2019-10-10,2019-10-15,6.0,85,3264.41,300,23,463.933332,408.483325,4263.106471,False
2,80001,0,-1,55,5,0,1,True,2019-12-01,2019-12-02,2.0,213,3151.01,462,18,278.050002,304.999999,5635.159203,False
3,80001,0,-1,55,5,0,1,True,2019-12-18,2019-12-20,3.0,23,3151.01,462,18,278.050002,304.999999,5635.159203,False
4,80001,0,-1,55,5,0,1,True,2019-12-27,2019-12-31,5.0,202,3151.01,462,18,278.050002,304.999999,5635.159203,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378462,123215,0,-1,0,9,21,0,False,2019-08-11,2019-08-15,5.0,19,0.00,0,0,0.000000,0.000000,0.000000,False
378463,177003,0,-1,0,9,21,0,False,2019-03-08,2019-03-08,1.0,175,0.00,0,0,0.000000,0.000000,0.000000,False
378464,180451,0,-1,0,9,21,0,False,2019-07-13,2019-07-25,13.0,1,0.00,0,0,0.000000,0.000000,0.000000,False
378465,156886,0,-1,33,5,20,2,True,2019-02-08,2019-02-19,12.0,12,937.50,19,8,102.116666,452.516668,308.483401,False


In [75]:
exp.groupby(['User_ID']).agg({'SMS_IN_CNT_M3' : [mean], })reset_index()

AttributeError: Cannot access callable attribute 'reset_index' of 'DataFrameGroupBy' objects, try using the 'apply' method